In [9]:
import sys
import numpy as np

sys.path.append("../src")

from nat2324.problems.sumplete import Sumplete
from nat2324.algorithms.ga import BinaryGeneticAlgorithm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
sumplete = Sumplete(3, evaluation_type="absolute")
ga = BinaryGeneticAlgorithm(p_m=0.01)

In [20]:
a = sumplete.evaluate(np.array([0, 1, 0, 0, 1, 0, 0, 0, 1]))
a

0

In [31]:
solution = ga.optimize(sumplete.evaluate)

Current best: N/A:   0%|          | 0/100 [00:00<?, ?it/s]/home/mantasu/projects/nat2324/notebooks/../src/nat2324/algorithms/ga.py:100: RuntimeWarning: invalid value encountered in divide
  p=fitness[~elites_mask] / fitness[~elites_mask].sum(),
Current best: N/A:   0%|          | 0/100 [00:00<?, ?it/s]


ValueError: probabilities contain NaN